In [1]:
import numpy as np
#import scipy.stats
#import seaborn as sns
import pandas as pd
import librosa
import librosa.display
from IPython.display import Audio, display
#import scikit_posthocs as sp
#import matplotlib.pyplot as plt
#from matplotlib import gridspec
#from matplotlib.transforms import BlendedGenericTransform
#import scikit_posthocs as sp
#from urllib.request import urlopen
#import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

import os

import keras
import keras.layers as layers
from keras import backend as K
import numpy as np

%matplotlib inline

Using TensorFlow backend.


TODO
* Dimensions are wrong - 
* Verify round trip to targets and back
* Debugging tools for comparing, listening to, and viewing clips

* Make evaluator that recreates a full clip
* Add audio comparisons between clean clips and cleaned clips
* Find a way to merge the input in at the last layer
* Check alignment between center of window and targets
* Consider edge behavior in terms of data windows (zeroes?)

* Change from tensor flow or investigate errors
* Confirm GPU use

* Evaluate different representations of complex numbers in terms of learnability



In [45]:
# some test data to hack around with
test_file = "Assets\\DataShareArchive\\Test\\Noisy\\p232_010.wav"
wav, rate = librosa.core.load(test_file)
fft = librosa.stft(wav)

In [3]:
# Constants and settings
WINDOW_SIZE = 10
FFT_BINS = 1025

In [35]:
#Utilities
def stack_fft(fft):
    return np.stack([fft.real, fft.imag], axis=-1)

def merge_fft_img(fft_img):
    return fft_img[:,:,0] + 1j * fft_img[:,:,1]

def unfold_target(t):
    return np.append(t.real, [t.imag])

def combine_target(t):
    return (t[0:FFT_BINS] + 1j * t[FFT_BINS:2*FFT_BINS]).reshape((1,FFT_BINS))

def targets_to_fft(targets):
    fft = np.empty((0,FFT_BINS))
    for i in range(0, targets.shape[0]):
        fft = np.concatenate([fft,combine_target(targets[i])])
    return fft.T  # swtich to (bins, samples) since model works in (samples, bins)
    

In [36]:
# Creating data from clip wave file
# output is (frames, window size, bins, layers), where image is depth 2 for real & imaginary
def clip_frames(file):
    wav, rate = librosa.core.load(file)
    half_win = WINDOW_SIZE//2
    fft = librosa.stft(wav) # organized as bins, frames
    pad = np.zeros((FFT_BINS,half_win))
    fft = np.concatenate([pad, fft, pad], axis=-1)
    frames = np.empty(shape=(0,FFT_BINS,WINDOW_SIZE,2)) 
    for i in range(0, fft.shape[1] - WINDOW_SIZE):
        frames = np.append(frames,[stack_fft(fft[:,i:i+WINDOW_SIZE])], axis=0)
    return frames

# output is array (samples, double bin length made up of real then imag)
def clip_targets(file):
    wav, rate = librosa.core.load(file)
    half_win = WINDOW_SIZE//2
    fft = librosa.stft(wav)
    pad = np.zeros((FFT_BINS,half_win))
    fft = np.concatenate([pad, fft, pad], axis=-1)
    targets = np.empty(shape=(0,2*FFT_BINS))
    for i in range(half_win, fft.shape[1] - half_win):
        targets = np.append(targets, [unfold_target(fft[:,i+half_win:i+half_win+1].flatten())], axis=0)
    return targets

In [46]:
ft = clip_frames(test_file)
tt = clip_targets(test_file)
print(ft.shape, tt.shape)

(120, 1025, 10, 2) (120, 2050)


In [47]:
new_fft = targets_to_fft(tt)

(1025, 120)


In [52]:
new_wav = librosa.istft(new_fft)
Audio(new_wav,rate=22050)

In [44]:
    
def show_fft(wav):
    librosa.display.specshow(librosa.amplitude_to_db(np.abs(librosa.stft(wav)), ref=np.max), y_axis='log', x_axis='time')

def display_fft(ft):
    librosa.display.specshow(librosa.amplitude_to_db(np.abs(fft), ref=np.max), y_axis='log', x_axis='time')

librosa.display.specshow(librosa.amplitude_to_db(np.abs(new_fft), ref=np.max), y_axis='log', x_axis='time')

print(new_fft.shape)

#show_fft(wav)
#show_fft(new_wav)
#Audio(wav, rate=22050)


ValueError: Window length M must be a non-negative integer

In [20]:
# Iterate over clean & noisy folders to create frames and targets
def create_data(root, max_files = 50):
    clean_dir = root + "\\Clean\\"
    noisy_dir = root + "\\Noisy\\"
    frames = np.empty(shape=(0,FFT_BINS,WINDOW_SIZE,2))
    targets = np.empty(shape=(0,2*FFT_BINS))
    for file in os.listdir(clean_dir)[0:max_files]:
        filename = os.fsdecode(file)
        frames = np.concatenate((frames,clip_frames(noisy_dir + file)), axis=0)
        targets = np.concatenate((targets,clip_targets(clean_dir + file)), axis=0)
    return frames, targets


In [21]:
frames, targets = create_data("Assets\\DataShareArchive\\Test", 10)

In [24]:
# Define network

model = keras.Sequential(
     [
        layers.Conv2D(32, kernel_size=2, activation='relu', padding='valid', 
                      input_shape=[FFT_BINS,WINDOW_SIZE,2]),
        layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
        layers.Conv2D(64, kernel_size=2, activation='relu'),
        layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
        layers.Flatten(),
        layers.Dense(FFT_BINS, activation='relu'),
        layers.Dense(150, activation='relu'),
        layers.Dense(50, activation='relu'),
        layers.Dense(2*FFT_BINS, activation='linear')
     ]
    )

model.compile(optimizer='Adam', loss='mse')

In [25]:
model.fit(frames, targets, epochs=5, batch_size=32)




Epoch 1/5





1832/1832 [==============================] - 16s 9ms/step - loss: 2.3977
Epoch 2/5
1832/1832 [==============================] - 14s 8ms/step - loss: 2.3964
Epoch 3/5
1832/1832 [==============================] - 14s 8ms/step - loss: 2.3963
Epoch 4/5
1832/1832 [==============================] - 14s 8ms/step - loss: 2.3962
Epoch 5/5
1832/1832 [==============================] - 14s 8ms/step - loss: 2.3960


In [ ]:
# rebuild clip
def clean_clip(n_file, model):
    n_frames = create_frames(n_file)
    slices = model.predict(n_frames)
    
    c_wav = librosa.istft(c_fft, 22050)
    targets = np.empty(shape=(0,2*FFT_BINS))
    for i in range(half_win, fft.shape[1] - half_win):
        targets = np.append(targets, [unfold_target(fft[:,i+half_win:i+half_win+1].flatten())], axis=0)
    return targets

In [27]:
p = model.predict(frames)

ft = np.empty((0,1025))
for i in range(0, p.shape[0]):
    ft = np.append(ft,[combine_target(p[i])], axis=0)

In [51]:
print(ft.shape)
w = librosa.istft(ft, 22050)
librosa.display.specshow(librosa.amplitude_to_db(np.abs(ft), ref=np.max), y_axis='log', x_axis='time')

(0, 1025)


OSError: [WinError 6] The handle is invalid

In [13]:
    a = np.array([[1+2j, 3+4j, 1.5+2.5j],[5+6j,7+8j, 5.5+6.5j]])
    print(a)
    a1 = a[0]
    print(a1)
    ar = a1.real
    ai = a1.imag
    t = np.append(ar, [ai])
    print (t)
    a2 = t[0:3] + 1j * t[3:6]
    print(a2)
    

[[1. +2.j  3. +4.j  1.5+2.5j]
 [5. +6.j  7. +8.j  5.5+6.5j]]
[1. +2.j  3. +4.j  1.5+2.5j]
[1.  3.  1.5 2.  4.  2.5]
[1. +2.j  3. +4.j  1.5+2.5j]


In [29]:
b

matrix([[2., 4.],
        [6., 8.],
        [1., 3.],
        [5., 7.]])